<a href="https://colab.research.google.com/github/dmartimarti/cancer_glucose/blob/master/smiles2morgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit-pypi

     |████████████████████████████████| 19.7 MB 1.3 MB/s 


In [22]:
# Importing libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np


#1. Load an excel file named biolog_metabolites_cancer.xlsx using Pandas
import pandas as pd
df = pd.read_excel('biolog_metabolites_cancer.xlsx')

#2. Remove NaN values from the column smiles
df = df.dropna(subset=['smiles'])

#3. Remove duplicates from the column smiles
df = df.drop_duplicates(subset=['smiles'])

#4. Convert a column of molecule smiles to the Morgan fingerprints using the library RDKit
from rdkit import Chem
from rdkit.Chem import AllChem

mols = [Chem.MolFromSmiles(x) for x in df['smiles']]
fps = [AllChem.GetMorganFingerprintAsBitVect(x,3,nBits=1024) for x in mols]

#5. Create a new dataframe, with name as df_drug, with the Drug column from the original df
df_drug = df[['Drug']]


In [23]:
#6. Convert the fingerprints as bits, and include them in the dataframe df_drug, each value as separate column in the dataframe
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray

ecfp6_name = [f'Bit_{i+1}' for i in range(1024)]
ecfp6_bits = [list(l) for l in fps]
df_drug = pd.DataFrame(ecfp6_bits, index = df_drug.Drug, columns=ecfp6_name)

#7. Save the dataframe as a csv file
df_drug.to_csv('drug_cancer_biolog_morganFP.csv', index=True)

In [4]:
df_drug

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,Bit_10,Bit_11,Bit_12,Bit_13,Bit_14,Bit_15,Bit_16,Bit_17,Bit_18,Bit_19,Bit_20,Bit_21,Bit_22,Bit_23,Bit_24,Bit_25,Bit_26,Bit_27,Bit_28,Bit_29,Bit_30,Bit_31,Bit_32,Bit_33,Bit_34,Bit_35,Bit_36,Bit_37,Bit_38,Bit_39,...,Bit_984,Bit_985,Bit_986,Bit_987,Bit_988,Bit_989,Bit_990,Bit_991,Bit_992,Bit_993,Bit_994,Bit_995,Bit_996,Bit_997,Bit_998,Bit_999,Bit_1000,Bit_1001,Bit_1002,Bit_1003,Bit_1004,Bit_1005,Bit_1006,Bit_1007,Bit_1008,Bit_1009,Bit_1010,Bit_1011,Bit_1012,Bit_1013,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
Drug,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Solasodine,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
Rotenone,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
Aklavine Hydrochloride,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Deguelin,0,1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Celastrol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doxorubicin Hydrochloride,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0
Etoposide,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
Nocodazole,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# 8. Calculate a t-SNE of df_drug with 3 dimensions
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=0, perplexity=30.0, 
            early_exaggeration=12.0, learning_rate=20, n_iter=10000)
tsne_obj = tsne.fit_transform(df_drug)

# 9. Save the t-SNE as a csv file
tsne_df = pd.DataFrame(tsne_obj, columns=['x', 'y', 'z'])
tsne_df.to_csv('drug_cancer_biolog_tsne_3D.csv', index=False)

# 10. Plot the t-SNE with Plotly in 3 dimensions, use the Drug names as labels
import plotly.express as px

fig = px.scatter_3d(tsne_df, x='x', y='y', z='z',
              color=df_drug.index,
              labels={'color':'Drug'})
fig.show()

In [7]:
# 8. Calculate a t-SNE of df_drug with 3 dimensions
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0, perplexity=30.0, 
            early_exaggeration=12.0, learning_rate=20, n_iter=10000)
tsne_obj = tsne.fit_transform(df_drug)

# 9. Save the t-SNE as a csv file
tsne_df = pd.DataFrame(tsne_obj, columns=['x', 'y'])
tsne_df.to_csv('drug_cancer_biolog_tsne_2D.csv', index=False)

# 10. Plot the t-SNE with Plotly in 3 dimensions, use the Drug names as labels
import plotly.express as px

fig = px.scatter(tsne_df, x='x', y='y',
              color=df_drug.index,
              labels={'color':'Drug'})
fig.show()

In [10]:
# 11. Calculate a PCA of the numeric variables of df_drug and plot it in 3D with Plotly
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_obj = pca.fit_transform(df_drug)

pca_df = pd.DataFrame(pca_obj, columns=['x', 'y', 'z'])
pca_df.to_csv('drug_cancer_biolog_pca.csv', index=False)

fig = px.scatter_3d(pca_df, x='x', y='y', z='z',
              color=df_drug.index,
              labels={'color':'Drug'})
fig.show()

In [12]:
# 11. Calculate a PCA of the numeric variables of df_drug and plot it in 3D with Plotly
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_obj = pca.fit_transform(df_drug)

pca_df = pd.DataFrame(pca_obj, columns=['PC1', 'PC2'])
pca_df.to_csv('drug_cancer_biolog_pca_2D.csv', index=False)

fig = px.scatter(pca_df, x='PC1', y='PC2',
              color=df_drug.index,
              labels={'color':'Drug'})
fig.show()

In [14]:
# 11. Calculate a PCA of the numeric variables of df_drug and plot it in 3D with Plotly
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
pca_obj = pca.fit_transform(df_drug)

pca_df = pd.DataFrame(pca_obj)

pca_df

,0,1,2,3,4,5,6,7,8,9
0,0.519326,0.281057,0.552024,-0.341603,-3.039654,0.099729,1.308868,-1.354679,1.435484,-1.260044
1,2.173961,1.453638,-0.507162,1.841415,-1.084544,3.643288,0.758246,0.987800,0.726701,-2.164364
2,4.676897,-4.460591,-2.051707,-1.029932,1.144342,-1.302395,-1.230694,0.321762,-0.851179,-0.658644
3,2.204124,1.185687,-0.369203,1.982899,-0.947193,3.376976,0.762512,0.867074,0.561308,-2.121485
4,-0.306085,0.100065,0.063231,0.216493,-1.594899,-1.050779,1.729238,-0.112024,0.001184,-1.385640
...,...,...,...,...,...,...,...,...,...,...
84,4.412099,-3.871413,-2.141424,-1.144767,0.596920,1.019684,0.156376,0.421986,1.190870,1.044572
85,4.609398,5.228735,-0.226584,-2.001959,1.132357,-2.149611,-0.499354,-0.945781,-1.189365,-0.423280
86,-0.240948,0.071931,0.527590,0.974322,1.293038,-0.651034,0.676308,-0.116071,0.703153,0.385766
87,0.184917,-0.103113,-0.952526,-0.570817,-0.809545,-0.898380,0.477031,-0.222526,1.077531,0.272840


In [ ]:
# 12. Calculate the clusters from the PCA and create a new dataframe with the cluster information
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=12, random_state=0)
kmeans_obj = kmeans.fit_predict(pca_df)

kmeans_df = pd.DataFrame(kmeans_obj, columns=['cluster'])
kmeans_df.to_csv('drug_cancer_biolog_pca_cluster.csv', index=False)

In [ ]:
# 13. Plot the PCA with Plotly with the cluster information
fig = px.scatter_3d(pca_df, x='x', y='y', z='z',
              color=kmeans_df.cluster,
              labels={'color':'Cluster'})
fig.show()


In [ ]:
# 14. Calculate the clusters from the t-SNE and create a new dataframe with the cluster information
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=12, random_state=0)
kmeans_obj = kmeans.fit_predict(tsne_df)

kmeans_df = pd.DataFrame(kmeans_obj, columns=['cluster'])
kmeans_df.to_csv('drug_cancer_biolog_tsne_cluster.csv', index=False)

# 15. Plot the t-SNE with Plotly with the cluster information
fig = px.scatter_3d(tsne_df, x='x', y='y', z='z',
              color=kmeans_df.cluster,
              labels={'color':'Cluster'})
fig.show()

In [ ]:
# 16. Calculate the clusters from the original dataframe and create a new dataframe with the cluster information
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0)
kmeans_obj = kmeans.fit_predict(df_drug)

kmeans_df = pd.DataFrame(kmeans_obj, columns=['cluster'])
kmeans_df.to_csv('drug_cancer_biolog_cluster.csv', index=False)



ValueError: ignored